#   Struct

###  Andrea Iskander Belkhir id : 511089

###  Beyza Özdemir id : 576145

In [10]:
using BenchmarkTools
using LinearAlgebra
using StaticArrays
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

   # Box(model)

 return [min,max], those parameter define the containment zone of the struct, if the parameter min return as 0 this mean we used the origin of the coordinate system  

In [2]:
square = Lar.cuboid([1,1])

([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])

In [3]:
x = Lar.Struct([square])

LinearAlgebraicRepresentation.Struct([([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])], [[0.0; 0.0], [1.0; 1.0]], "2872412809724459662", 2, "feature")

In [4]:
table = Lar.apply(Lar.t(-0.5,-0.5),square)

([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]])

In [5]:
y = Lar.Struct([repeat([table,Lar.r(pi/2)],outer=2)...])

LinearAlgebraicRepresentation.Struct(Any[([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0], ([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0]], [[-0.5; -0.5], [0.5; 0.5]], "13297476787215158445", 2, "feature")

In [7]:
function box(model)
	if isa(model,Matrix)
		return nothing
	elseif isa(model,Lar.Struct)
		listOfModels = Lar.evalStruct(model)
		#dim = checkStruct(listOfModels)
		if listOfModels == []
			return model.box
		else
			theMin,theMax = box(listOfModels[1])
			for theModel in listOfModels[2:end]
				modelMin,modelMax = box(theModel)
				for (k,val) in enumerate(modelMin)
					if val < theMin[k]
						theMin[k]=val
					end
				end
				for (k,val) in enumerate(modelMax)
					if val > theMax[k]
						theMax[k]=val
					end
				end
			end
		end
		return [theMin,theMax]

	elseif (isa(model,Tuple) ||isa(model,Array))&& (length(model)>=2)
		V = model[1]
		theMin = minimum(V, dims=2)
		theMax = maximum(V, dims=2)
	end

	return [theMin,theMax]
end

box (generic function with 1 method)

In [7]:
@btime box(x)

  5.217 μs (49 allocations: 2.53 KiB)


2-element Vector{Matrix{Float64}}:
 [0.0; 0.0]
 [1.0; 1.0]

In [8]:
@btime box(y)

  13.500 μs (110 allocations: 5.17 KiB)


2-element Vector{Matrix{Float64}}:
 [-0.5; -0.5]
 [0.5; 0.5]

In [9]:
@benchmark box(y)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  10.900 μs …  6.523 ms  ┊ GC (min … max): 0.00% … 98.65%
 Time  (median):     11.800 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   14.146 μs ± 65.310 μs  ┊ GC (mean ± σ):  4.55% ±  0.99%

  ▆█▇▅▃▅▄▂▁▂▁▂▂▂▁    ▁▁▁▁                                     ▂
  █████████████████▇██████▇▇▇▇▆▆▆▇▇▅▆▅▅▄▅▅▅▅▅▅▅▆▄▅▄▅▅▄▄▃▄▄▅▂▄ █
  10.9 μs      Histogram: log(frequency) by time      36.3 μs <

 Memory estimate: 5.17 KiB, allocs estimate: 110.

# Improvment and changes

In [6]:
@inline function box(model)
	if (isa(model,MMatrix) || isa(model,Matrix))
		return nothing
	elseif isa(model,Lar.Struct)
		listOfModels = Lar.evalStruct(model)
		#dim = checkStruct(listOfModels)
		if listOfModels == []
			return model.box
		else
			theMin,theMax = evalBox(listOfModels)
		end
        
		return [theMin,theMax]

	elseif (isa(model,Tuple) || isa(model,Array)) && (length(model)>=2)
		V = model[1]
		theMin = minimum(V, dims=2)
		theMax = maximum(V, dims=2)
	end
    
	return [theMin,theMax]
end

box (generic function with 1 method)

In [22]:
@inline function evalBox(listOfModels)
	theMin,theMax = box(listOfModels[1])
	for theModel in listOfModels[2:end]
		modelMin,modelMax= box(theModel)
		@async @inbounds for (k,val) in enumerate(modelMin)
			if val < theMin[k]
				theMin[k]=val
			end
		end
		@async @inbounds for (k,val) in enumerate(modelMax)
			if val > theMax[k]
				theMax[k]=val
			end
		end
	end
	return theMin,theMax
end

evalBox (generic function with 1 method)

In [23]:
@benchmark box(y)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  14.300 μs … 192.000 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     28.400 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   32.386 μs ±  12.352 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▁    ▁▆██▆▆▇▅▄▄▄▄▄▂▂▂▁▂▂▂▂▁▁                               ▂
  █▅█▆██▇██████████████████████████▇▇█▇▇▇▇▇▇█▆▆▆▇▆▆▆▆▆▆▅▇▅▅▄▄▆ █
  14.3 μs       Histogram: log(frequency) by time      87.2 μs <

 Memory estimate: 5.52 KiB, allocs estimate: 96.

piu lento ma deve funzionare fare il check anche su mmatrix

In [15]:
@btime box(y)

  14.100 μs (96 allocations: 5.52 KiB)


2-element Vector{Matrix{Float64}}:
 [-0.5; -0.5]
 [0.5; 0.5]

In [24]:
@btime box(x)

  5.617 μs (49 allocations: 2.53 KiB)


2-element Vector{Matrix{Float64}}:
 [0.0; 0.0]
 [1.0; 1.0]

In [25]:
x.box

2-element Vector{Matrix{Float64}}:
 [0.0; 0.0]
 [1.0; 1.0]